In [1]:
import bs4
import time
from datetime import datetime
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
from selenium import webdriver


In [2]:
def get_soup_objects(html):
    soup = BeautifulSoup(html,"html.parser")
    return soup

In [3]:
def get_medium_html(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content,"html.parser")
    time.sleep(3) 
    return soup

In [4]:
def get_data_medium_web(soup):
    articles=soup.find_all("article")
    
    # Declaration
    articles_topic = []
    articles_content = []
    
    for article in articles:
        # Get article's topic
        topic = article.find("h2").get_text()
        articles_topic.append(topic)

        # Get article's content
        content = article.find('a')['href']
        url_of_article = "https://medium.com/" + article.find('a')['href']
        soup = get_medium_html(url_of_article)
        content = soup.find("main").find_all("p")
        
        full_content = ""
        if content != None:
            for paragraph in content:
                full_content = full_content + " " + paragraph.get_text().strip()
        articles_content.append(full_content)
      
    return articles_topic, articles_content

In [5]:
def insert_data_to_csv(topic, content):
    # Create dataframe
    data = pd.DataFrame({
    "topic": topic,
    #"category": category,
    "content": content
    })
    data.to_csv(f'Medium Scraping-{datetime.now().strftime("%d-%m-%Y")}.csv', mode='a')

In [6]:
categories = ["technology", "health", "business", "money", "entrepreneurship", "marketing", "life"]
print("Start scraping from Medium.com!")
driver = webdriver.Chrome()
driver.get(f"https://medium.com/tag/{category")
driver.maximize_window()
for i in range (1, 15):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
html = driver.page_source
driver.close()
soup_obj = get_soup_objects(html)
topics, contents = get_data_medium_web(soup_obj)
insert_data_to_csv(topics, contents)
print("Finish scraping from Medium.com!")

In [ ]:
categories = ["technology", "health", "business", "money", "entrepreneurship", "marketing", "life"]
print("Start scraping from Medium.com!")
for category in categories:
    soup = get_medium_html(f"https://medium.com/tag/{category}")
    topic, content = get_data_medium_web(soup)
    insert_data_to_csv(topic, category, content)
print("Finish scraping from Medium.com!")